In [3]:
# 安装PaddleOCR环境
!git clone https://gitee.com/paddlepaddle/PaddleOCR -b release/2.1
!cd PaddleOCR && pip install -r requirements.txt
!cd PaddleOCR && python setup.py install

fatal: destination path 'PaddleOCR' already exists and is not an empty directory.
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.0 MB 20.7 MB/s eta 0:00:01
     |████████████████████████████████| 12.5 MB 8.9 MB/s eta 0:00:011
     |████████████████████████████████| 948 kB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 299 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 34.2 MB 8.5 MB/s eta 0:00:012
     |████████████████████████████████| 165 kB 42.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 14.5 MB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=171679 sha256=3adc3992d1c120cbd09b2781f48e4aa07b926649c5f5d676d59ec8ed3e790d5c
  Stored in directory: /home/aistudio/.cache/pip/whe

In [2]:
# 安装PaddleSlim环境
!git clone https://gitee.com/paddlepaddle/PaddleSlim -b release/2.0.0
!cd PaddleSlim && pip install -r requirements.txt
!cd PaddleSlim && python setup.py install

fatal: destination path 'PaddleSlim' already exists and is not an empty directory.
Looking in indexes: https://mirror.baidu.com/pypi/simple/
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
[04-27 10:56:57 MainThread @logger.py:242] Argv: setup.py install
[04-27 10:56:57 Main

In [ ]:
# 解压数据集
!unzip /home/aistudio/data/data87683/训练数据集.zip -d /home/aistudio/data/train_images
!unzip /home/aistudio/data/data87685/A榜测试数据集.zip -d /home/aistudio/data/test_images

  inflating: /home/aistudio/data/train_images/__

In [4]:
# 预处理数据集标签
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
def Q2B(s):
    """全角转半角"""
    inside_code=ord(s)
    if inside_code==0x3000:
        inside_code=0x0020
    else:
        inside_code-=0xfee0
    if inside_code<0x0020 or inside_code>0x7e: #转完之后不是半角字符返回原来的字符
        return s
    return chr(inside_code)

def stringQ2B(s):
    """把字符串全角转半角"""
    return "".join([Q2B(c) for c in s])

def is_chinese(s):
    """判断unicode是否是汉字"""
    for c in s:
        if c < u'\u4e00' or c > u'\u9fa5':
            return False
    return True

def is_number(s):
    """判断unicode是否是数字"""
    for c in s:
        if c < u'\u0030' or c > u'\u0039':
            return False
    return True

def is_alphabet(s):
    """判断unicode是否是英文字母"""
    for c in s:
        if (u'\u0041'<= c <= u'\u005a') or (u'\u0061'<= c <= u'\u007a'):
            return True
    return False

def del_other(s):
    """判断是否非汉字，数字和英文"""
    res = str()
    for c in s:
        if not (is_chinese(c) or is_number(c) or is_alphabet(c)):
            c = ""
        res += c
    return res

# 原始label文件
src_label_file = "/home/aistudio/data/train_images/训练数据集/LabelTrain.txt"
dst_label_file = "/home/aistudio/work/label.txt"
label_list_file = "/home/aistudio/work/label_dict.txt"
src_data_root = "/home/aistudio/data/train_images/训练数据集/TrainImages"

m_width = 0
m_height = 0
label_max_len = 0
width_dict = dict()
height_dict = dict()
widths = list()
heights = list()
classSet = set()

with open(src_label_file,'r', encoding= 'utf-8') as src_label, \
     open(dst_label_file, "w", encoding= 'utf-8') as dst_label:

    lines = src_label.readlines()

    for line in lines:
        path, label = line.split("\t")
        img = Image.open(os.path.join(src_data_root, path))
        width, height = img.size
        
        # 全角转半角
        label = stringQ2B(label.split('\n')[0])
        # # 删除特殊符号
        # label = del_other(label)

        # 记录数据集中最长标签
        if len(label) > label_max_len:
            label_max_len = len(label)

        if label != "":
            dst_label.write(path + "\t" + label + "\n")
        
        for e in label:
            classSet.add(e)
            
        widths.append(width)
        heights.append(height)
        width_dict[int(width)] = 1 if (int(width)) not in width_dict else 1 + width_dict[int(width)]
        height_dict[int(height)] = 1 if (int(height)) not in height_dict else 1 + height_dict[int(height)]
        m_width += width
        m_height += height

m_width = m_width/len(lines)
m_height = m_height/len(lines)

aspect_ratio = m_width/m_height

width_dict = dict(sorted(width_dict.items(), key=lambda item: item[1], reverse=True))
height_dict = dict(sorted(height_dict.items(), key=lambda item: item[1], reverse=True))

print("aspect ratio is: {}, mean width is: {}, mean height is: {}".format(aspect_ratio,m_width,m_height))
print("Width dict:{}".format(width_dict))
print("Height dict:{}".format(height_dict))
print("Label max len:{}".format(label_max_len))

plt.hist(widths,bins = len(width_dict))
plt.show()
plt.hist(heights,bins = len(height_dict))
plt.show()

with open(label_list_file, "w", encoding= 'utf-8') as label_list:
    classList = sorted(list(classSet))
    for idx, c in enumerate(classList):
        label_list.write("{}\n".format(c))

print("classify num: ", len(classSet))
print("Train label save in {}".format(dst_label_file))
print("Train list save in {}".format(label_list_file))

aspect ratio is: 4.389499976829576, mean width is: 161.9747, mean height is: 36.90049
Width dict:{51: 759, 48: 743, 52: 740, 50: 718, 40: 693, 37: 689, 49: 688, 53: 686, 44: 683, 32: 682, 41: 681, 46: 678, 38: 678, 39: 673, 43: 667, 34: 666, 30: 657, 47: 653, 45: 652, 28: 648, 35: 646, 54: 645, 29: 642, 26: 627, 33: 627, 63: 625, 36: 621, 58: 618, 27: 617, 57: 613, 62: 607, 31: 604, 61: 603, 56: 602, 42: 601, 60: 599, 55: 595, 65: 581, 64: 571, 66: 569, 59: 568, 67: 564, 74: 551, 71: 548, 69: 547, 68: 542, 72: 536, 25: 533, 75: 527, 79: 526, 73: 524, 70: 510, 78: 489, 87: 488, 24: 487, 77: 487, 76: 477, 88: 476, 80: 474, 86: 474, 82: 471, 84: 470, 90: 467, 83: 459, 81: 456, 92: 453, 89: 446, 96: 442, 85: 441, 93: 434, 94: 434, 23: 431, 91: 428, 22: 422, 99: 421, 98: 421, 97: 416, 107: 402, 100: 399, 95: 395, 102: 389, 103: 384, 101: 382, 104: 375, 113: 373, 21: 369, 110: 364, 105: 360, 108: 356, 112: 354, 114: 352, 20: 348, 111: 344, 106: 344, 115: 344, 120: 342, 119: 340, 109: 340, 11

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

classify num:  3937
Train label save in /home/aistudio/work/label.txt
Train list save in /home/aistudio/work/label_dict.txt


In [4]:
import random
import os
import shutil
# 按8：2将数据集划分为训练集和验证集
ratio = 0.8
seed = 12

dst_train_root = "/home/aistudio/data/rec/train"
dst_eval_root = "/home/aistudio/data/rec/eval"

if not os.path.exists(dst_train_root):
    os.makedirs(dst_train_root)

if not os.path.exists(dst_eval_root):
    os.makedirs(dst_eval_root)

print(dst_label_file)
dst_train_label_file = "/home/aistudio/data/rec/rec_gt_train.txt"
dst_eval_label_file = "/home/aistudio/data/rec/rec_gt_eval.txt"

with open(dst_label_file, "r", encoding= 'utf-8') as label_file:
    lines = label_file.readlines()
    random.seed(seed)
    random.shuffle(lines)
    train_lines = lines[0:int(ratio*len(lines))]
    eval_lines = lines[int(ratio*len(lines)):len(lines)]

    # 创建训练集
    for idx, line in enumerate(train_lines):
        path, label = line.split("\t")
        shutil.copy(os.path.join(src_data_root,path), os.path.join(dst_train_root,path))
        train_lines[idx] = line.replace("img", "rec/train/img")

    # 创建验证集
    for idx, line in enumerate(eval_lines):
        path, label = line.split("\t")
        shutil.copy(os.path.join(src_data_root,path), os.path.join(dst_eval_root,path))
        eval_lines[idx] = line.replace("img", "rec/eval/img")


with open(dst_train_label_file, "w", encoding= 'utf-8') as train_label_file, \
     open(dst_eval_label_file, "w", encoding= 'utf-8') as eval_label_file:
    train_label_file.writelines(train_lines)
    eval_label_file.writelines(eval_lines)

/home/aistudio/work/label.txt


In [2]:
!cp /home/aistudio/work/ppocr/modeling/necks/rnn.py /home/aistudio/PaddleOCR/ppocr/modeling/necks/rnn.py
!cp /home/aistudio/work/tools/train.py /home/aistudio/PaddleOCR/tools/train.py
!cp /home/aistudio/work/tools/infer_rec.py /home/aistudio/PaddleOCR/tools/infer_rec.py

In [8]:
!python /home/aistudio/PaddleOCR/tools/train.py -c /home/aistudio/work/configs/rec_mobilev3_train.yml

[2021/04/26 23:35:26] root INFO: Architecture : 
[2021/04/26 23:35:26] root INFO:     Backbone : 
[2021/04/26 23:35:26] root INFO:         layers : 18
[2021/04/26 23:35:26] root INFO:         name : ResNet
[2021/04/26 23:35:26] root INFO:     Head : 
[2021/04/26 23:35:26] root INFO:         fc_decay : 4e-05
[2021/04/26 23:35:26] root INFO:         name : CTCHead
[2021/04/26 23:35:26] root INFO:     Neck : 
[2021/04/26 23:35:26] root INFO:         encoder_type : rnn
[2021/04/26 23:35:26] root INFO:         hidden_size : 128
[2021/04/26 23:35:26] root INFO:         name : SequenceEncoder
[2021/04/26 23:35:26] root INFO:     Transform : None
[2021/04/26 23:35:26] root INFO:     algorithm : CRNN
[2021/04/26 23:35:26] root INFO:     model_type : rec
[2021/04/26 23:35:26] root INFO: Eval : 
[2021/04/26 23:35:26] root INFO:     dataset : 
[2021/04/26 23:35:26] root INFO:         data_dir : ./data
[2021/04/26 23:35:26] root INFO:         label_file_list : ['./data/rec/rec_gt_eval.txt']
[2021/0

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 